In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('air_temp.csv')
df.columns = ['year','month','day','time','time_zone','temperature']
df.head()

,year,month,day,time,time_zone,temperature
0,2021,6,1,00:00,UTC,6.2
1,2021,6,1,01:00,UTC,6.4
2,2021,6,1,02:00,UTC,6.4
3,2021,6,1,03:00,UTC,6.8
4,2021,6,1,04:00,UTC,7.1


In [3]:
## concatenate "year","month" and "day" to creat a new column "date"
if 'date' in df.columns:
    df.drop(['date'],axis=1,inplace=True)
time_stamp = df["year"].astype(str)+'-'+df["month"].astype(str)+'-'+df["day"].astype(str)
df.insert(0,"date",time_stamp)
df.head()

,date,year,month,day,time,time_zone,temperature
0,2021-6-1,2021,6,1,00:00,UTC,6.2
1,2021-6-1,2021,6,1,01:00,UTC,6.4
2,2021-6-1,2021,6,1,02:00,UTC,6.4
3,2021-6-1,2021,6,1,03:00,UTC,6.8
4,2021-6-1,2021,6,1,04:00,UTC,7.1


## Alternative solutions of A1.6

### solution 1
Using dataframe.pivot() method to reshape the dataframe, and then select different columns for feature and label

In [4]:
df_s1 = df[df['time'].isin(['01:00','11:00','12:00'])] # filter "time" of interest

df_s1 = df_s1.pivot(index="date", columns="time", values="temperature")

df_s1.dropna(axis=0,inplace=True) #drop rows with missing record

df_s1.head()

time,01:00,11:00,12:00
date,,,
2021-6-1,6.4,14.1,16.5
2021-6-10,9.3,19.0,18.4
2021-6-11,11.2,24.4,24.8
2021-6-12,12.4,25.4,26.6
2021-6-13,11.6,9.0,9.7


In [5]:
X = df_s1['11:00'].to_numpy().reshape(-1,1)  
y = df_s1['12:00'].to_numpy()

# Sanity check to help you detect major mistakes
assert np.isclose(X[0,0],14.1), 'Feature matrix is incorrect'
assert X.shape == (91,1), 'The shape of feature matrix is incorrect'
assert y.shape == (91,), 'The shape of label vector is incorrect'

### solution 2
Slice records from the raw dataframe based on some conditions

In [6]:
## check if there are days associated missing record for a certain time, if there are, drop them
df_s2 = df.sort_values( by = ['date', 'time']) #sort the dataframe based on the value of column 'date' and 'time'
df_s2 = df_s2[df_s2['time'].isin(["11:00","12:00"])]
dates = df_s2['date'].unique() 
for date in dates:
    datapoint = df_s2[(df_s2['date']==date)] 
    if sum(datapoint.isnull().any()) !=0:
        print("This day is associated with missing record, we should remove it:",date)
        print("\n",df_s2[df_s2['date']==date]) # you will see "NaN" entry

This day is associated with missing record, we should remove it: 2021-6-18

           date  year  month  day   time time_zone  temperature
419  2021-6-18  2021      6   18  11:00       UTC          NaN
420  2021-6-18  2021      6   18  12:00       UTC         16.8


In [7]:
df_s2.drop(df_s2[df_s2['date']=="2021-6-18"].index, inplace=True) # remove the "date" associated missing record

In [8]:
X = df_s2[df_s2.time=='11:00'].temperature.to_numpy().reshape(-1,1)
y = df_s2[df_s2.time=='12:00'].temperature.to_numpy()

# Sanity check to help you detect major mistakes
assert np.isclose(X[0,0],14.1), 'Feature matrix is incorrect'
assert X.shape == (91,1), 'The shape of feature matrix is incorrect'
assert y.shape == (91,), 'The shape of label vector is incorrect'

### solution 3
This script represents a typical way to preprocess time-series data for machine learning problems. First, the date and time stamps are organized into the index with proper frequency. Then, the features and labels are obtained via indexing. We reveal the missing values that would otherwise go unnoticed in the process.

Using primarily Pandas, we take advantage of its Numpy-based and optimized built-in methods. Furthermore, the code here is much more compact and readable than Python loops.

In [9]:
# create a new column that combines date and time
df['datetime'] = df['date'] + ' ' + df['time']
# convert the column into the datetime64 Dtype: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M")
# verifying the Dtype of 'datetime' 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2209 entries, 0 to 2208
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2209 non-null   object        
 1   year         2209 non-null   int64         
 2   month        2209 non-null   int64         
 3   day          2209 non-null   int64         
 4   time         2209 non-null   object        
 5   time_zone    2209 non-null   object        
 6   temperature  2204 non-null   float64       
 7   datetime     2209 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 138.2+ KB


In [10]:
# remove the unnecessary 'date' and 'time' columns, leave only 'datetime' and 'temperature'
df_s3 = df[['datetime', 'temperature']]
# setting a datetime64 column as the index gives us a wide range of indexing capabilities
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#indexing
df_s3.set_index("datetime",inplace=True)
# since we know this is hourly data, setting the frequency accordingly will reveal all the
# missing values
df_s3 = df_s3.asfreq('H')
df_s3.head()

,temperature
datetime,
2021-06-01 00:00:00,6.2
2021-06-01 01:00:00,6.4
2021-06-01 02:00:00,6.4
2021-06-01 03:00:00,6.8
2021-06-01 04:00:00,7.1


In [11]:
df_s3[df_s3.isnull().any(axis=1)]  # check if any missing data

,temperature
datetime,
2021-06-05 09:00:00,NaN
2021-06-05 19:00:00,NaN
2021-06-18 07:00:00,NaN
2021-06-18 08:00:00,NaN
2021-06-18 11:00:00,NaN


In [12]:
# constructing X and y vectors.
X = df_s3[df_s3.index.hour == 11]
# transforming the datetime index of X and y for compatibility
X.index = X.index.date
y = df_s3[df_s3.index.hour == 12]
y.index = y.index.date
X.head()

,temperature
2021-06-01,14.1
2021-06-02,25.1
2021-06-03,28.2
2021-06-04,22.3
2021-06-05,9.7


In [13]:
# concatenating the two Dataframes using their indices
res = pd.concat([X,y], axis=1)
# dropping the row which contains the missing value
res.columns = ["mintemp","maxtemp"]
res.dropna(inplace=True)
res.head()

,mintemp,maxtemp
2021-06-01,14.1,16.5
2021-06-02,25.1,25.5
2021-06-03,28.2,27.1
2021-06-04,22.3,23.9
2021-06-05,9.7,10.7


In [14]:
X = res.iloc[:,0].to_numpy().reshape(-1,1)
y = res.iloc[:,1].to_numpy()

# Sanity check to help you detect major mistakes
assert np.isclose(X[0,0],14.1), 'Feature matrix is incorrect'
assert X.shape == (91,1), 'The shape of feature matrix is incorrect'
assert y.shape == (91,), 'The shape of label vector is incorrect'